In [ ]:
import tags
import pid
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
vcap = cv2.VideoCapture("AprilTagTest.mkv")

In [ ]:
PIDVertical = pid.PID(0.09,0,0,100)
PIDHorizontal = pid.PID(0.05,0,0,100)
id_to_track = 0

In [ ]:
frequency = 1
maxRetry = 1000
count = 0

In [ ]:
out = cv2.VideoWriter('april_tag_render.mp4', cv2.VideoWriter_fourcc(*"mp4v"), 20.0, (1920,1080))

while count < maxRetry:
    ret, img = vcap.read()

    if not ret:
        break

    print(f"now on frame {count}…")

    if count % frequency == 0:
        temp = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        apriltags = tags.get_tags(temp)
        tag_img = tags.draw_tags(apriltags, img)

        centers = tags.get_positions(apriltags)
        relative_centers = tags.error_relative_to_center(centers, img.shape[0], img.shape[1])
        
        powY = 0
        powX = 0

        for center in relative_centers:
            if center[2] == 0:
                powY= PIDHorizontal.update(center[0])
                powX = PIDVertical.update(center[1])
        
        print(f"Vertical Thrusters: {powX}, Horizontal Thrusters: {powY}")

        out.write(tag_img)

        # plt.imshow(tag_img)
        # plt.show()

        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break

    count += 1

print("Render completed.")
vcap.release()
out.release()

RELEASE EVERYTHING IF JOB IS DONE